電影資料庫   判正頻論 或 負評論

In [1]:
%matplotlib inline

import numpy as np
import matplotlib.pyplot as plt

In [2]:
# 匯入影評
from keras.datasets import imdb

Using TensorFlow backend.


In [3]:
(X_train , Y_train),(X_test , Y_test)=imdb.load_data(num_words=10000) #只要一萬字常用的

In [4]:
print("訓練資料的總比數:",len(X_train))
print("訓練資料的總比數:",len(X_test))

訓練資料的總比數: 25000
訓練資料的總比數: 25000


In [5]:
#X_train[24999]   #一坨向量  出現最多次的字 數字小 1號    多次 數字大

In [6]:
len(X_train[25])

142

In [7]:
len(X_train[55])

766

不同長度

In [8]:
Y_train[25]  

1

In [9]:
Y_train[:10]  #前十筆    

array([1, 0, 0, 1, 0, 0, 1, 0, 1, 0], dtype=int64)

輸入 是一個文章(很多字)   輸出 一個值

送入神經網路的輸入處理
雖然 RNN 是可以處理不同長度的輸入, 在寫程式時我們還是要

設輸入文字長度的上限
把每段文字都弄成一樣長, 太短的後面補上 0

In [10]:
from keras.preprocessing import sequence   #幫我們先處理字數

In [11]:
X_train=sequence.pad_sequences(X_train,maxlen=250)  #我們訂150個字
X_test=sequence.pad_sequences(X_test,maxlen=250)

## 決定神經網路架構
####  -先將 10000 維的文字壓到 N 維
#### -然後用 K 個 LSTM 神經元做隱藏層
#### -最後一個 output, 直接用 sigmoid 送出

In [12]:
N=33
K=43


In [13]:
from keras.models import Sequential
from keras.layers import Dense ,Embedding ,Dropout  #Embedding 壓縮用
from keras.layers import LSTM

In [14]:
model=Sequential()

In [15]:
model.add(Embedding(10000,N))  #壓成三維
model.add(Dropout(0.2))

In [16]:
model.add(LSTM(43))  #input 不用寫了

In [17]:
model.add(Dense(units=32,activation="relu"))
model.add(Dropout(0.2))
model.add(Dense(1,activation="sigmoid"))

In [18]:
# 組裝

In [19]:
model.compile(loss="binary_crossentropy",
             optimizer="adam"
             ,metrics=["acc"])

In [20]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (None, None, 33)          330000    
_________________________________________________________________
dropout_1 (Dropout)          (None, None, 33)          0         
_________________________________________________________________
lstm_1 (LSTM)                (None, 43)                13244     
_________________________________________________________________
dense_1 (Dense)              (None, 32)                1408      
_________________________________________________________________
dropout_2 (Dropout)          (None, 32)                0         
_________________________________________________________________
dense_2 (Dense)              (None, 1)                 33        
Total params: 344,685
Trainable params: 344,685
Non-trainable params: 0
_________________________________________________________________


In [21]:
# 一個LSTM有 4 cell 一個cell 有7個input + b   
# so 有 4*7+4=32個 全重要調
# 32 *4 =128個

In [22]:
model.fit(X_train,Y_train,batch_size=32,epochs=4)

Epoch 1/4
25000/25000 [==============================] - 348s 14ms/step - loss: 0.4128 - acc: 0.8112
Epoch 2/4
25000/25000 [==============================] - 369s 15ms/step - loss: 0.2542 - acc: 0.9023
Epoch 3/4
25000/25000 [==============================] - 323s 13ms/step - loss: 0.1979 - acc: 0.9269
Epoch 4/4
25000/25000 [==============================] - 322s 13ms/step - loss: 0.1630 - acc: 0.9396


In [24]:
score = model.evaluate(X_test, Y_test)

25000/25000 [==============================] - 103s 4ms/step


In [25]:
print(f'測試資料的 loss = {score[0]}')
print(f'測試資正確率 = {score[1]}')

測試資料的 loss = 0.3510954908466339
測試資正確率 = 0.86176


In [26]:
model_json = model.to_json()
open('imdb_model_arch.json',
     'w').write(model_json)
model.save_weights('imdb_model_weights.h5')